In [40]:
import pandas as pd
import requests
import numpy as np
import statsapi

In [41]:
url = 'http://statsapi.mlb.com/api/v1/schedule/games/?sportId=1'
r = requests.get(url=url).json()
dates = r['dates'][0]['games']
df = pd.DataFrame(dates)
df

,gamePk,gameGuid,link,gameType,season,gameDate,officialDate,status,teams,venue,...,seasonDisplay,dayNight,description,scheduledInnings,reverseHomeAwayStatus,inningBreakLength,seriesDescription,recordSource,ifNecessary,ifNecessaryDescription
0,747298,e5f968e3-58cb-4137-b71a-e7ded3dd5387,/api/v1.1/game/747298/feed/live,A,2024,2024-07-17T00:00:00Z,2024-07-16,"{'abstractGameState': 'Preview', 'codedGameSta...","{'away': {'leagueRecord': {'wins': 0, 'losses'...","{'id': 5325, 'name': 'Globe Life Field', 'link...",...,2024,night,"94th All-Star Game, in Arlington, TX",9,False,120,All-Star Game,S,N,Normal Game


In [42]:
HOST = 'https://statsapi.mlb.com'
url = HOST + df.loc[0,'link']
r = requests.get(url=url).json()
player_df = pd.DataFrame(r['gameData']['players'])
player_df_all = pd.DataFrame(['players'])
player_df = player_df.T

In [43]:
player = statsapi.player_stat_data(694973, group= "hitting", type="season", sportId=1)
# float(player['stats'][0]['stats']['avg'])
# print(player)

def get_avg(player_id):
    player = statsapi.player_stat_data(int(player_id), group= "hitting", type="season", sportId=1)
    if player['position'] == 'P':
        return 0
    return float(player['stats'][0]['stats']['avg'])

averages = player_df.id.apply(get_avg)
player_df = player_df.assign(avg=averages.values)

SSLError: HTTPSConnectionPool(host='statsapi.mlb.com', port=443): Max retries exceeded with url: /api/v1/people/683002?hydrate=stats(group=hitting,type=season,sportId=1),currentTeam (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))

In [ ]:
strikezone_size = player_df.strikeZoneTop - player_df.strikeZoneBottom
player_df = player_df.assign(strikezone_size=strikezone_size.values)

In [ ]:
not_pitchers = player_df['avg'] != 0
ax = player_df[not_pitchers].plot.scatter(x='avg', y='strikezone_size', alpha = 0.3)

In [ ]:
player = statsapi.player_stat_data(502110, group= "hitting", type="season", sportId=1)
float(player['stats'][0]['stats']['avg'])
print(player)

In [ ]:
def get_babip(player_id):
    player = statsapi.player_stat_data(int(player_id), group= "hitting", type="season", sportId=1)
    if player['position'] == 'P':
        return 0
    return float(player['stats'][0]['stats']['babip'])

def get_num_pitches(player_id):
    player = statsapi.player_stat_data(int(player_id), group= "hitting", type="season", sportId=1)
    if player['position'] == 'P':
        return 0
    return float(player['stats'][0]['stats']['numberOfPitches'])

def get_homeruns(player_id):
    player = statsapi.player_stat_data(int(player_id), group= "hitting", type="season", sportId=1)
    if player['position'] == 'P':
        return 0
    return float(player['stats'][0]['stats']['homeRuns'])

player_babip = player_df.id.apply(get_babip)
player_df = player_df.assign(babip=player_babip.values)

player_num_pitches = player_df.id.apply(get_num_pitches)
player_df = player_df.assign(num_pitches=player_num_pitches.values)

player_homeruns = player_df.id.apply(get_homeruns)
player_df = player_df.assign(homeruns=player_homeruns.values)

player_df

In [ ]:
ax = player_df[not_pitchers].plot.scatter(x='weight', y='babip', alpha = 0.3)

In [ ]:
ax = player_df[not_pitchers].plot.scatter(x='num_pitches', y='avg', alpha = 0.3)

In [ ]:
ax = player_df[not_pitchers].plot.scatter(x='babip', y='avg', alpha = 0.5, color = 'red')

In [ ]:
ax = player_df[not_pitchers].plot.scatter(x='homeruns', y='avg', alpha = 0.5, color = 'red')

In [ ]:
ax = player_df[not_pitchers].plot.hist(column='avg', bins = 20)
ax.set_xlabel('Batting Average')

In [ ]:
ax = player_df[not_pitchers].plot.bar(x='primaryPosition', y='avg')
ax.set_ylabel('Batting Average')
ax.set_ylabel('Primary Position')

In [ ]:
ax = player_df[not_pitchers].plot.scatter(x='age', y='avg', alpha = 0.5, color = 'red')
a